# Experimentación - Parte 2: Aplicación a DynamicWal
Cargamos data Walmart, corremos baseline y 3 configs de NN para predicción demanda. Métricas: MSE, Revenue Error.

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Fallback load_and_preprocess_data (tu fallback)
def load_and_preprocess_data(target_col='quantity_sold'):
    df = pd.read_csv("../data/Walmart.csv")  # Ajusta ruta si necesario (o "data/Walmart.csv" si cwd = root)
    feature_cols = ['unit_price', 'holiday_indicator', 'store_id']
    X = df[feature_cols].fillna(0).values
    y = df[target_col].values.reshape(-1, 1)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    return (X_train, y_train), (X_test, y_test), scaler

# Fallback business_metric
def business_metric(y_true, y_pred, base_price=100):
    revenue_true = y_true * base_price
    revenue_pred = y_pred * base_price
    return np.mean(np.abs(revenue_true - revenue_pred)) / np.mean(revenue_true) * 100

# Fallback NeuralNetwork inline (tu código original + losses fix)
class NeuralNetwork:
    def __init__(self, layers, activation='relu'):
        self.layers = layers
        self.activation_name = activation
        self.weights = []
        self.biases = []

        for i in range(len(layers) - 1):
            n_in = layers[i]
            n_out = layers[i + 1]

            if activation.lower() == 'relu':
                weight = np.random.randn(n_in, n_out) * np.sqrt(2.0 / n_in)
            else:
                weight = np.random.randn(n_in, n_out) * np.sqrt(1.0 / n_in)

            bias = np.zeros((1, n_out))

            self.weights.append(weight)
            self.biases.append(bias)

        if activation.lower() == 'relu':
            self.activation = self.relu
            self.activation_derivative = self.relu_derivative
        elif activation.lower() == 'sigmoid':
            self.activation = self.sigmoid
            self.activation_derivative = self.sigmoid_derivative
        elif activation.lower() == 'tanh':
            self.activation = self.tanh
            self.activation_derivative = self.tanh_derivative
        else:
            raise ValueError(f"Función de activación desconocida: {activation}")

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        s = self.sigmoid(x)
        return s * (1 - s)

    def tanh(self, x):
        return np.tanh(x)

    def tanh_derivative(self, x):
        return 1 - np.tanh(x)**2

    def forward(self, X):
        self.a = [X]
        self.z = []
        for w, b in zip(self.weights[:-1], self.biases[:-1]):
            z = np.dot(self.a[-1], w) + b
            self.z.append(z)
            self.a.append(self.activation(z))
        z = np.dot(self.a[-1], self.weights[-1]) + self.biases[-1]
        self.z.append(z)
        self.a.append(z)
        return z

    def backward(self, y_true):
        m = y_true.shape[0]
        dz = (self.a[-1] - y_true) / m
        self.dW = []
        self.db = []

        for i in reversed(range(len(self.weights))):
            a_prev = self.a[i]
            dW = np.dot(a_prev.T, dz)
            db = np.sum(dz, axis=0, keepdims=True)
            self.dW.insert(0, dW)
            self.db.insert(0, db)
            if i != 0:
                dz = np.dot(dz, self.weights[i].T) * self.activation_derivative(self.z[i-1])

    def train(self, X, y, epochs=100, learning_rate=0.01):
        losses = []
        for epoch in range(epochs):
            self.forward(X)
            loss = np.mean((self.a[-1] - y)**2) / 2
            losses.append(loss)
            self.backward(y)
            for i in range(len(self.weights)):
                self.weights[i] -= learning_rate * self.dW[i]
                self.biases[i] -= learning_rate * self.db[i]
        return losses

    def predict(self, X):
        return self.forward(X)

# Carga data
(X_train, y_train), (X_test, y_test), _ = load_and_preprocess_data()

# Baseline
model = LinearRegression()
model.fit(X_train, y_train)
y_pred_bl = model.predict(X_test)
mse_bl = mean_squared_error(y_test, y_pred_bl)
rev_bl = business_metric(y_test.flatten(), y_pred_bl.flatten())
print(f"Baseline MSE: {mse_bl:.4f}, Rev Error: {rev_bl:.2f}%")

# 3 Configs de NN
configs = [
    ([8], "relu", 0.01, 300),
    ([6], "sigmoid", 0.01, 300),
    ([12, 6], "tanh", 0.01, 300)
]
for hidden, act, lr, eps in configs:
    nn = NeuralNetwork([X_train.shape[1]] + hidden + [1], activation=act)
    losses = nn.train(X_train, y_train, epochs=eps, learning_rate=lr)
    y_pred = nn.predict(X_test).flatten()
    mse = mean_squared_error(y_test, y_pred)
    rev = business_metric(y_test.flatten(), y_pred)
    print(f"Config {hidden}-{act}: MSE {mse:.4f}, Rev Error {rev:.2f}%")

# Análisis negocio
print("\nAnálisis: Rev Error <45% viable para DynamicWal (uplift potencial 3%).")

Baseline MSE: 2.0328, Rev Error: 39.82%
Config [8]-relu: MSE 2.1212, Rev Error 40.98%
Config [6]-sigmoid: MSE 2.0877, Rev Error 40.73%
Config [12, 6]-tanh: MSE 2.0428, Rev Error 39.97%

Análisis: Rev Error <45% viable para DynamicWal (uplift potencial 3%).


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Fallback load_and_preprocess_data
def load_and_preprocess_data(target_col='quantity_sold'):
    df = pd.read_csv("../data/Walmart.csv")  # Cambia a "data/Walmart.csv" si cwd = root
    feature_cols = ['unit_price', 'holiday_indicator', 'store_id']
    X = df[feature_cols].fillna(0).values
    y = df[target_col].values.reshape(-1, 1)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    return (X_train, y_train), (X_test, y_test), scaler

# Fallback business_metric
def business_metric(y_true, y_pred, base_price=100):
    revenue_true = y_true * base_price
    revenue_pred = y_pred * base_price
    return np.mean(np.abs(revenue_true - revenue_pred)) / np.mean(revenue_true) * 100

# Fallback NeuralNetwork inline (tu código original + losses fix)
class NeuralNetwork:
    def __init__(self, layers, activation='relu'):
        self.layers = layers
        self.activation_name = activation
        self.weights = []
        self.biases = []

        for i in range(len(layers) - 1):
            n_in = layers[i]
            n_out = layers[i + 1]

            if activation.lower() == 'relu':
                weight = np.random.randn(n_in, n_out) * np.sqrt(2.0 / n_in)
            else:
                weight = np.random.randn(n_in, n_out) * np.sqrt(1.0 / n_in)

            bias = np.zeros((1, n_out))

            self.weights.append(weight)
            self.biases.append(bias)

        if activation.lower() == 'relu':
            self.activation = self.relu
            self.activation_derivative = self.relu_derivative
        elif activation.lower() == 'sigmoid':
            self.activation = self.sigmoid
            self.activation_derivative = self.sigmoid_derivative
        elif activation.lower() == 'tanh':
            self.activation = self.tanh
            self.activation_derivative = self.tanh_derivative
        else:
            raise ValueError(f"Función de activación desconocida: {activation}")

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        s = self.sigmoid(x)
        return s * (1 - s)

    def tanh(self, x):
        return np.tanh(x)

    def tanh_derivative(self, x):
        return 1 - np.tanh(x)**2

    def forward(self, X):
        self.a = [X]
        self.z = []
        for w, b in zip(self.weights[:-1], self.biases[:-1]):
            z = np.dot(self.a[-1], w) + b
            self.z.append(z)
            self.a.append(self.activation(z))
        z = np.dot(self.a[-1], self.weights[-1]) + self.biases[-1]
        self.z.append(z)
        self.a.append(z)
        return z

    def backward(self, y_true):
        m = y_true.shape[0]
        dz = (self.a[-1] - y_true) / m
        self.dW = []
        self.db = []

        for i in reversed(range(len(self.weights))):
            a_prev = self.a[i]
            dW = np.dot(a_prev.T, dz)
            db = np.sum(dz, axis=0, keepdims=True)
            self.dW.insert(0, dW)
            self.db.insert(0, db)
            if i != 0:
                dz = np.dot(dz, self.weights[i].T) * self.activation_derivative(self.z[i-1])

    def train(self, X, y, epochs=100, learning_rate=0.01):
        losses = []
        for epoch in range(epochs):
            self.forward(X)
            loss = np.mean((self.a[-1] - y)**2) / 2
            losses.append(loss)
            self.backward(y)
            for i in range(len(self.weights)):
                self.weights[i] -= learning_rate * self.dW[i]
                self.biases[i] -= learning_rate * self.db[i]
        return losses

    def predict(self, X):
        return self.forward(X)

# Carga data
(X_train, y_train), (X_test, y_test), _ = load_and_preprocess_data()

# Baseline
model = LinearRegression()
model.fit(X_train, y_train)
y_pred_bl = model.predict(X_test)
mse_bl = mean_squared_error(y_test, y_pred_bl)
rev_bl = business_metric(y_test.flatten(), y_pred_bl.flatten())
print(f"Baseline MSE: {mse_bl:.4f}, Rev Error: {rev_bl:.2f}%")

# 3 Configs de NN
configs = [
    ([8], "relu", 0.01, 300),
    ([6], "sigmoid", 0.01, 300),
    ([12, 6], "tanh", 0.01, 300)
]
for hidden, act, lr, eps in configs:
    nn = NeuralNetwork([X_train.shape[1]] + hidden + [1], activation=act)
    losses = nn.train(X_train, y_train, epochs=eps, learning_rate=lr)
    y_pred = nn.predict(X_test).flatten()
    mse = mean_squared_error(y_test, y_pred)
    rev = business_metric(y_test.flatten(), y_pred)
    print(f"Config {hidden}-{act}: MSE {mse:.4f}, Rev Error {rev:.2f}%")

# Análisis negocio
print("\nAnálisis: Rev Error <45% viable para DynamicWal (uplift potencial 3%).")

Baseline MSE: 2.0328, Rev Error: 39.82%
Config [8]-relu: MSE 2.1492, Rev Error 41.23%
Config [6]-sigmoid: MSE 2.0575, Rev Error 40.24%
Config [12, 6]-tanh: MSE 2.0438, Rev Error 40.11%

Análisis: Rev Error <45% viable para DynamicWal (uplift potencial 3%).
